# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [14]:
# imports
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

load_dotenv()

True

In [15]:
# constants

MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"
OLLAMA_URL = "http://localhost:11434/api/chat"
client = OpenAI()

In [16]:
def technical_question_model(model_type, quest):
    model_type = model_type.strip().lower()

    system_prompt = "You are a senior engineer. Explain clearly with steps and examples."

    # --- OpenAI ---
    if model_type == "openai":
        corresponding_model = MODEL_GPT

        response = client.chat.completions.create(
            model=corresponding_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": quest},
            ],
            temperature=0.2,
        )
        return response.choices[0].message.content

    # --- Ollama ---
    elif model_type == "ollama":
        corresponding_model = MODEL_LLAMA

        r = requests.post(
            OLLAMA_URL,
            json={
                "model": corresponding_model,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": quest},
                ],
                "stream": False,
            },
            timeout=120,
        )
        r.raise_for_status()
        data = r.json()
        return data["message"]["content"]

    else:
        raise ValueError("model_type must be 'openai' or 'ollama'")

In [ ]:
# Get gpt-4o-mini to answer, with streaming
display(Markdown(technical_question_model("openai", "Explain binary search with an example.")))


In [ ]:
display(Markdown(technical_question_model("ollama", "Explain what CORS is.")))